In [1]:
# Set local python and nltk paths
import sys
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/IPython/extensions')
sys.path

['/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/IPython/extensions',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages',
 '',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python27.zip',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/plat-linux2',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-tk',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-old',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-dynload',
 '/homes/ag6516/.local/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python

In [2]:
%matplotlib inline

import os
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
from random import randint
random.seed(123)
np.random.seed(123)

# Set THEANO_FLAGS='device=cuda0,floatX=float32' to run notebook on gpu
import lasagne
from lasagne.utils import floatX
import theano
import theano.tensor as T
from collections import Counter
from lasagne.utils import floatX

Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 1050 (0000:01:00.0)


In [3]:
dir = ('/vol/medic02/users/ag6516/x_ray_fracture_localisation/')
# dir = ('/Users/Aydan/PhD/x_ray_fracture_localisation/')

In [23]:
sample_images = pickle.load(open('sample_images_baseline.pkl'))
print len(sample_images)

1432


In [24]:
allwords = Counter()
for item in sample_images:
    tokens = item['tokens']
    #for sentence in item['sentences']:
    allwords.update(tokens)
        
vocab = [k for k, v in allwords.items()]
vocab.insert(0, '#START#')
vocab.append('#END#')

word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for i, w in enumerate(vocab)}

len(vocab)

107

In [25]:
exam_ids = []
for folder in os.listdir(dir+'data/Images'):
    #os.path.exists(self.labelfilename)
    exam_ids.append(str(folder))

In [26]:
print len(exam_ids)

330


## Max Pool CNN image embeddings

In [27]:
pids = [item['patient id'] for item in sample_images]
pids = set(pids)
print len(pids)

327


In [28]:
sample_exams = []
for pid in exam_ids:
    if pid in pids:
        all_cnn_features = [item['cnn features'] for item in sample_images if item['patient id']==pid]
        exam = [item for item in sample_images if item['patient id'] == pid][0]
        all_cnn_features_array = np.array(all_cnn_features)
        exam['max_cnn_features'] = np.max(all_cnn_features_array, axis=0)
        sample_exams.append(exam)
    else:
        continue

In [29]:
pickle.dump(sample_exams, open('sample_exams_cnn_pooled_features.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)

# Build Image Caption Model
<img src="caption_model.png">

Implemented in Theano/Lasagne based on __[pydata2015 tutorial](https://github.com/ebenolson/pydata2015/tree/master/4%20-%20Recurrent%20Networks)__ for natural image captioning

In [30]:
sample_exams = pickle.load(open('sample_exams_cnn_pooled_features.pkl'))
print len(sample_exams)

327


In [31]:
SEQUENCE_LENGTH = 33
MAX_SENTENCE_LENGTH = SEQUENCE_LENGTH - 3 # 1 for image, 1 for start token, 1 for end token
#BATCH_SIZE = 50
BATCH_SIZE = 10
CNN_FEATURE_SIZE = 1024
EMBEDDING_SIZE = 256

In [32]:
# Returns a list of tuples (cnn features, list of words, image ID)
def get_data_batch(dataset, size, split='train'):
    items = []
    
    while len(items) < size:
        item = random.choice(dataset)
        if item['split'] != split:
            continue
        sentence = item['tokens']
        if len(sentence) > MAX_SENTENCE_LENGTH:
            sentence = sentence[1:MAX_SENTENCE_LENGTH]
            #continue
        items.append((item['max_cnn_features'], sentence, item['patient id']))
    
    return items

In [33]:
# Convert a list of tuples into arrays that can be fed into the network
def prep_batch_for_network(batch):
    x_cnn = floatX(np.zeros((len(batch), CNN_FEATURE_SIZE)))
    x_sentence = np.zeros((len(batch), SEQUENCE_LENGTH - 1), dtype='int32')
    y_sentence = np.zeros((len(batch), SEQUENCE_LENGTH), dtype='int32')
    mask = np.zeros((len(batch), SEQUENCE_LENGTH), dtype='bool')

    for j, (cnn_features, sentence, _) in enumerate(batch):
        x_cnn[j] = cnn_features
        i = 0
        for word in ['#START#'] + sentence + ['#END#']:
            if word in word_to_index:
                mask[j, i] = True
                y_sentence[j, i] = word_to_index[word]
                x_sentence[j, i] = word_to_index[word]
                i += 1
        #mask[j, 0] = False
                
    return x_cnn, x_sentence, y_sentence, mask

In [34]:
# sentence embedding maps integer sequence with dim (BATCH_SIZE, SEQUENCE_LENGTH - 1) to 
# (BATCH_SIZE, SEQUENCE_LENGTH-1, EMBEDDING_SIZE)
l_input_sentence = lasagne.layers.InputLayer((BATCH_SIZE, SEQUENCE_LENGTH - 1))
l_sentence_embedding = lasagne.layers.EmbeddingLayer(l_input_sentence,
                                                     input_size=len(vocab),
                                                     output_size=EMBEDDING_SIZE,
                                                    )

# cnn embedding changes the dimensionality of the representation from 1024 to EMBEDDING_SIZE, 
# and reshapes to add the time dimension - final dim (BATCH_SIZE, 1, EMBEDDING_SIZE)
l_input_cnn = lasagne.layers.InputLayer((BATCH_SIZE, CNN_FEATURE_SIZE))
l_cnn_embedding = lasagne.layers.DenseLayer(l_input_cnn, num_units=EMBEDDING_SIZE,
                                            nonlinearity=lasagne.nonlinearities.identity)

l_cnn_embedding = lasagne.layers.ReshapeLayer(l_cnn_embedding, ([0], 1, [1]))

# the two are concatenated to form the RNN input with dim (BATCH_SIZE, SEQUENCE_LENGTH, EMBEDDING_SIZE)
l_rnn_input = lasagne.layers.ConcatLayer([l_cnn_embedding, l_sentence_embedding])


l_dropout_input = lasagne.layers.DropoutLayer(l_rnn_input, p=0.5)

l_lstm = lasagne.layers.LSTMLayer(l_dropout_input,
                                  num_units=EMBEDDING_SIZE,
                                  unroll_scan=True,
                                  grad_clipping=5.)

l_dropout_output = lasagne.layers.DropoutLayer(l_lstm, p=0.5)

# the RNN output is reshaped to combine the batch and time dimensions
# dim (BATCH_SIZE * SEQUENCE_LENGTH, EMBEDDING_SIZE)
l_shp = lasagne.layers.ReshapeLayer(l_dropout_output, (-1, EMBEDDING_SIZE))

# decoder is a fully connected layer with one output unit for each word in the vocabulary
l_decoder = lasagne.layers.DenseLayer(l_shp, num_units=len(vocab), nonlinearity=lasagne.nonlinearities.softmax)

# finally, the separation between batch and time dimension is restored
l_out = lasagne.layers.ReshapeLayer(l_decoder, (BATCH_SIZE, SEQUENCE_LENGTH, len(vocab)))

In [35]:
# cnn feature vector
x_cnn_sym = T.matrix()

# sentence encoded as sequence of integer word tokens
x_sentence_sym = T.imatrix()

# mask defines which elements of the sequence should be predicted
mask_sym = T.imatrix()

# ground truth for the RNN output
y_sentence_sym = T.imatrix()

In [36]:
output = lasagne.layers.get_output(l_out, {
                l_input_sentence: x_sentence_sym,
                l_input_cnn: x_cnn_sym
                })

In [37]:
def calc_cross_ent(net_output, mask, targets):
    # Helper function to calculate the cross entropy error
    preds = T.reshape(net_output, (-1, len(vocab)))
    targets = T.flatten(targets)
    cost = T.nnet.categorical_crossentropy(preds, targets)[T.flatten(mask).nonzero()]
    return cost

loss = T.mean(calc_cross_ent(output, mask_sym, y_sentence_sym))

In [38]:
MAX_GRAD_NORM = 15

all_params = lasagne.layers.get_all_params(l_out, trainable=True)

all_grads = T.grad(loss, all_params)
all_grads = [T.clip(g, -5, 5) for g in all_grads]
all_grads, norm = lasagne.updates.total_norm_constraint(
    all_grads, MAX_GRAD_NORM, return_norm=True)

updates = lasagne.updates.adam(all_grads, all_params, learning_rate=0.001)

f_train = theano.function([x_cnn_sym, x_sentence_sym, mask_sym, y_sentence_sym],
                          [loss, norm],
                          updates=updates
                         )

f_val = theano.function([x_cnn_sym, x_sentence_sym, mask_sym, y_sentence_sym], loss)

In [39]:
for iteration in range(100000):
    x_cnn, x_sentence, y_sentence, mask = prep_batch_for_network(get_data_batch(sample_exams, BATCH_SIZE))
    loss_train, norm = f_train(x_cnn, x_sentence, mask, y_sentence)
    if not iteration % 250:
        print('Iteration {}, loss_train: {}, norm: {}'.format(iteration, loss_train, norm))
        try:
            batch = get_data_batch(sample_exams, BATCH_SIZE, split='val')
            x_cnn, x_sentence, y_sentence, mask = prep_batch_for_network(batch)
            loss_val = f_val(x_cnn, x_sentence, mask, y_sentence)
            print('Val loss: {}'.format(loss_val))
        except IndexError:
            continue

Iteration 0, loss_train: 4.66846942902, norm: 0.817887842655
Val loss: 4.5886926651
Iteration 250, loss_train: 2.31142282486, norm: 0.739516079426
Val loss: 1.75672996044
Iteration 500, loss_train: 2.25775527954, norm: 1.05124402046
Val loss: 1.20261228085
Iteration 750, loss_train: 1.264980793, norm: 0.964316725731
Val loss: 0.895298361778
Iteration 1000, loss_train: 1.33384406567, norm: 1.20700645447
Val loss: 1.02093696594
Iteration 1250, loss_train: 1.23437571526, norm: 1.17228233814
Val loss: 0.853420734406
Iteration 1500, loss_train: 1.09603011608, norm: 1.30654478073
Val loss: 0.881110846996
Iteration 1750, loss_train: 0.908065080643, norm: 1.029281497
Val loss: 0.789424955845
Iteration 2000, loss_train: 0.840369462967, norm: 1.05683624744
Val loss: 1.04973554611
Iteration 2250, loss_train: 0.981441140175, norm: 1.20361673832
Val loss: 0.84905731678
Iteration 2500, loss_train: 0.653212845325, norm: 0.832866668701
Val loss: 0.909407734871
Iteration 2750, loss_train: 0.68497937917

Iteration 23000, loss_train: 0.311948567629, norm: 0.627546429634
Val loss: 0.76567709446
Iteration 23250, loss_train: 0.292551994324, norm: 0.40446922183
Val loss: 0.753806769848
Iteration 23500, loss_train: 0.3649738729, norm: 0.83100181818
Val loss: 0.751128077507
Iteration 23750, loss_train: 0.338958829641, norm: 1.37261664867
Val loss: 0.88844782114
Iteration 24000, loss_train: 0.273982435465, norm: 0.340812444687
Val loss: 0.775251567364
Iteration 24250, loss_train: 0.281099826097, norm: 0.40333122015
Val loss: 0.865962684155
Iteration 24500, loss_train: 0.317500829697, norm: 0.350274980068
Val loss: 0.704030692577
Iteration 24750, loss_train: 0.373214423656, norm: 0.424488067627
Val loss: 0.827071428299
Iteration 25000, loss_train: 0.432173937559, norm: 0.777556836605
Val loss: 0.759003281593
Iteration 25250, loss_train: 0.294577658176, norm: 0.245822653174
Val loss: 0.797252714634
Iteration 25500, loss_train: 0.372259646654, norm: 0.523230671883
Val loss: 0.721362054348
Iterati

Iteration 45750, loss_train: 0.37110888958, norm: 0.626123845577
Val loss: 0.754658699036
Iteration 46000, loss_train: 0.408224493265, norm: 0.993426382542
Val loss: 0.668911337852
Iteration 46250, loss_train: 0.415863156319, norm: 1.19144368172
Val loss: 0.693383812904
Iteration 46500, loss_train: 0.304676175117, norm: 0.233259916306
Val loss: 0.823782324791
Iteration 46750, loss_train: 0.298696041107, norm: 0.345302313566
Val loss: 0.814334988594
Iteration 47000, loss_train: 0.40731459856, norm: 0.560810387135
Val loss: 0.790430903435
Iteration 47250, loss_train: 0.377935647964, norm: 0.31066557765
Val loss: 0.794009268284
Iteration 47500, loss_train: 0.360856980085, norm: 0.294677108526
Val loss: 0.780932307243
Iteration 47750, loss_train: 0.331382542849, norm: 0.633453309536
Val loss: 0.691099762917
Iteration 48000, loss_train: 0.318385690451, norm: 0.739931404591
Val loss: 0.719907164574
Iteration 48250, loss_train: 0.355801641941, norm: 0.470222175121
Val loss: 0.755940675735
Ite

Iteration 68500, loss_train: 0.323946177959, norm: 1.1880825758
Val loss: 0.78806167841
Iteration 68750, loss_train: 0.318909525871, norm: 0.344785064459
Val loss: 0.745927512646
Iteration 69000, loss_train: 0.319739252329, norm: 0.91435700655
Val loss: 0.736110508442
Iteration 69250, loss_train: 0.451804757118, norm: 4.69880819321
Val loss: 0.761819124222
Iteration 69500, loss_train: 0.442890673876, norm: 3.99261903763
Val loss: 0.760464191437
Iteration 69750, loss_train: 0.394435942173, norm: 1.54993927479
Val loss: 0.723653972149
Iteration 70000, loss_train: 0.335575968027, norm: 2.77436304092
Val loss: 0.723720669746
Iteration 70250, loss_train: 0.306185692549, norm: 0.426524877548
Val loss: 0.816475212574
Iteration 70500, loss_train: 0.36001098156, norm: 0.78386503458
Val loss: 0.721702754498
Iteration 70750, loss_train: 0.298007667065, norm: 0.994714319706
Val loss: 0.749407887459
Iteration 71000, loss_train: 0.298131942749, norm: 0.874167740345
Val loss: 0.755921721458
Iteration

Iteration 91500, loss_train: 0.435743242502, norm: 8.4769487381
Val loss: 0.83804833889
Iteration 91750, loss_train: 0.304664522409, norm: 1.57559072971
Val loss: 0.719573974609
Iteration 92000, loss_train: 0.387373358011, norm: 6.04626131058
Val loss: 0.821368396282
Iteration 92250, loss_train: 0.371883928776, norm: 3.20156478882
Val loss: 0.876356601715
Iteration 92500, loss_train: 0.343702346087, norm: 1.22084391117
Val loss: 0.76218354702
Iteration 92750, loss_train: 0.423362135887, norm: 1.29959630966
Val loss: 0.704079687595
Iteration 93000, loss_train: 0.407871246338, norm: 1.71606814861
Val loss: 0.819805443287
Iteration 93250, loss_train: 0.357493877411, norm: 0.726890683174
Val loss: 0.719980895519
Iteration 93500, loss_train: 0.29124262929, norm: 1.47849178314
Val loss: 0.716050028801
Iteration 93750, loss_train: 0.293184131384, norm: 1.65610635281
Val loss: 0.751059651375
Iteration 94000, loss_train: 0.380883336067, norm: 2.22458028793
Val loss: 0.748826503754
Iteration 942

In [40]:
param_values = lasagne.layers.get_all_param_values(l_out)
d = {'param values': param_values,
     'vocab': vocab,
     'word_to_index': word_to_index,
     'index_to_word': index_to_word,
    }
pickle.dump(d, open('maxpooling_trained_v_107.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)